# 신경망 학습
- 목적: 손실함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는 것


In [7]:
# SSE (오차제곱합)
import numpy as np
def sum_squares_error(y, t):
    return 0.5 * np.sum((y - t)**2)

In [1]:
# CEE (교차 엔트로피)
def cross_entropy_error(y, t):
    delta = 1e-7    # 로그에 0이 들어가면 안되니까
    return -np.sum(t * np.log(y + delta))

### 미니배치 학습
- 모든 데이터에 대한 손실 함수의 합을 다 구하려면 시간이 오래 걸림
- 데이터 일부를 추려 전체의 근사치로 이용 -> mini batch

In [5]:
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


# MNIST 데이터 불러오기
(x_train, t_train), (x_test, t_test) = mnist.load_data()

# --- normalize=True ---
# 픽셀 값을 [0,1] 범위로 변환
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# --- one_hot_label=True ---
# 정수 레이블(0~9)을 one-hot 인코딩
t_train = to_categorical(t_train, num_classes=10)
t_test = to_categorical(t_test, num_classes=10)
x_train = x_train.reshape(x_train.shape[0], -1)
x_test  = x_test.reshape(x_test.shape[0], -1)

print("훈련 데이터:", x_train.shape, t_train.shape)
print("테스트 데이터:", x_test.shape, t_test.shape)

훈련 데이터: (60000, 784) (60000, 10)
테스트 데이터: (10000, 784) (10000, 10)


In [8]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [9]:
np.random.choice(60000,10)

array([29629, 38123,  5967, 53562, 53151, 34347, 42968, 44693, 20561,
       11341], dtype=int32)

In [10]:
# 원핫인코딩 (배치용) 교차 엔트로피 오차 구하기
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [11]:
# 숫자 레이블로 주어졌을 때 (배치용) 교차 엔트로피 오차 구하기
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

### 수치 미분
- 고딩 때 배운 미분은 수치 미분
    - 이를 엄밀히 따지면 x와 x+h 사이의 기울기가 됨
- 진정한 접선을 구하기 위해 함수의 차분 도입
    - (x-h) + (x+h)/2h

In [12]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [13]:
def function_1(x):
    return -0.01*x**2 + 0.1*x

In [14]:
# 편미분
def function_2(x): 
    return x[0]**2 + x[1]**2

# x0=3, x1=4일 때 x0의 편미분 구하기
def function_tmp1(x0):
    return x0*x0 + 4.0**2

numerical_diff(function_tmp1, 3.0)

6.00000000000378

- 기울기: 모든 변수의 편미분을 벡터로 정리한 것

In [31]:
import numpy as np

def numerical_gradient(f, x):
    h = 1e-4  # 0.0001
    grad = np.zeros_like(x)

    # np.nditer를 사용하여 모든 차원의 요소를 안전하게 순회
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index  # 다차원 인덱스 (예: (0, 1))를 얻음
        
        # 원본 값 저장 후 약간의 변화를 줌
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)
        
        x[idx] = float(tmp_val) - h
        fxh2 = f(x)
        
        # 편미분 계산
        grad[idx] = (fxh1 - fxh2) / (2 * h)
        
        x[idx] = tmp_val  # 원본 값으로 복원
        
        it.iternext()  # 다음 요소로 이동
        
    return grad

In [16]:
numerical_gradient(function_2, np.array([3.0, 4.0]))

array([6., 8.])

- 위 결과가 의미하는 바는 기울기 그림이 있듬
- 기울기 그림은 방향을 가진 벡터(화살표)는 함수의 최솟값을 가리킴
- **기울기가 가리키는 쪽은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향** 

### 경사하강법
- 기울기를 이용해서 함수의 최솟값을 찾으려는 기법
- 기울기: 각 지점에서 함수의 값을 낮추는 방안을 제시하는 지표
- 그러나 기울기가 가리키는 곳에 정말 함수의 최솟값이 있는지 보장 X  

1. 현 위치에서 기울어진 방향으로 일정 거리만큼 이동
2. 이동한 곳에서도 기울기를 구하고 기울어진 방향으로 나아가길 반복
- 이 함수의 값을 점차 줄여가는 기법이 경사법
- 학습률: 한 번의 학습으로 얼마나 학습할지, 매개변수 값을 얼마나 갱신하는지 정하는 것

In [17]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x

    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

In [18]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

- 학습률이 너무 크면 발산
- 학습률이 너무 작으면 0이 안됨

In [19]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)   # 오버플로를 막기 위함
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y 

In [20]:
import sys, os

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [28]:
net = simpleNet()
print(net.W)

x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

print(np.argmax(p))

t = np.array([0, 0, 1]) # 정답 레이블
net.loss(x, t)

[[ 0.25343684  0.35021319  0.00181282]
 [-0.73577111 -1.14089259  1.23751323]]
[-0.5101319  -0.81667542  1.1148496 ]
2


np.float64(6.063616353282374)

In [32]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

print(dW)

[[-0.93584944 -0.4055888   1.34143825]
 [-1.40377416 -0.60838321  2.01215737]]


### SGD (확률적 경사 하강법)
- 지금까지 진행한 방식이 SGD
- 경사 하강법으로 매개변수를 갱신하는데, 데이터를 미니배치로 무작위로 선정하기 때문

In [33]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [46]:
# 2층 신경망 클래스 구현
import sys, os

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)

        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])   # float(x.shape[0]) 이 모야
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params["W1"])
        grads['b1'] = numerical_gradient(loss_W, self.params["b1"])
        grads['W2'] = numerical_gradient(loss_W, self.params["W2"])
        grads['b2'] = numerical_gradient(loss_W, self.params["b2"])

        return grads        # 기울기를 보관하는 거 

In [47]:
# 미니배치 학습
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical


# MNIST 데이터 불러오기
(x_train, t_train), (x_test, t_test) = mnist.load_data()

# --- normalize=True ---
# 픽셀 값을 [0,1] 범위로 변환
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# --- one_hot_label=True ---
# 정수 레이블(0~9)을 one-hot 인코딩
t_train = to_categorical(t_train, num_classes=10)
t_test = to_categorical(t_test, num_classes=10)
x_train = x_train.reshape(x_train.shape[0], -1)
x_test  = x_test.reshape(x_test.shape[0], -1)

train_loss_list = []

# hyperparameter
iters_num = 10000   # 반복횟수
train_size = x_train.shape[0]
batch_size = 100    # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)   # 28*28이미지 784, 0~9사이로 출력돼서 10

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ("W1", "b1", "W2", "b2"):
        network.params[key] - learning_rate * grad[key]

    # 학습 결과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

KeyboardInterrupt: 